<a href="https://colab.research.google.com/github/iamogbz/colab-research-gemini/blob/main/research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Gemini: Connecting to Gemini

The Gemini API allows you to connect to Google's most powerful multi-modal model. This example configures your API key and sends an example message to the API and prints a response.

Before you start, visit https://aistudio.google.com/app/apikey to create an API key.

In [43]:
# @title Configure Notebook internal parameters

gemini_api_secret_name = 'GOOGLE_API_KEY'  # @param {type: "string"}

# variables used in notebook
research_steps_raw = "[]" # json string output of ai generating research steps
step_ai_outputs = dict() # the results of the AI assisting with each step

# research step output format
research_step_key_step_number = "step_number"
research_step_key_step_description = "step_description"
research_step_key_ai_prompt = "ai_prompt_for_solving"

# research step current step structure format
current_step_key_reprompt = "reprompt"
current_step_key_value = "value"

# user editable parameters
system_instructions = 'You are a helpful research assistant' # @param {type: "string"}
model = 'gemini-1.5-flash' # @param {type: "string"} ["gemini-1.0-pro", "gemini-1.5-pro", "gemini-1.5-flash"]
temperature = 0.5 # @param {type: "slider", min: 0, max: 2, step: 0.05}
topic = "Automated Investigation and Research Assistants - Towards AI-powered Knowledge Discovery"  # @param {type: "string"}
step_count = 10 # @param {"type":"slider","min":5,"max":15,"step":1}

In [4]:
# @title Configure Gemini API key

import google.generativeai as genai
from google.colab import userdata

try:
  GOOGLE_API_KEY=userdata.get(gemini_api_secret_name)
  genai.configure(api_key=GOOGLE_API_KEY)
except userdata.SecretNotFoundError as e:
   print(f'Secret not found\n\nThis expects you to create a secret named {gemini_api_secret_name} in Colab\n\nVisit https://aistudio.google.com/app/apikey to create an API key\n\nStore that in the secrets section on the left side of the notebook (key icon)\n\nName the secret {gemini_api_secret_name}')
   raise e
except userdata.NotebookAccessError as e:
  print(f'You need to grant this notebook access to the {gemini_api_secret_name} secret in order for the notebook to access Gemini on your behalf.')
  raise e
except Exception as e:
  print(f"There was an unknown error. Ensure you have a secret {gemini_api_secret_name} stored in Colab and it's a valid key from https://aistudio.google.com/app/apikey")
  raise e

In [44]:
# @title Generate Steps For Researching Topic

import json

prompt = f"""
Generate a detailed list of {step_count} steps a researcher should take to create a satisfactory response paper on the topic: "{topic}".
The steps should be organized.
Include considerations for literature review, methodology, data collection (if applicable), analysis, and writing.
Only output the steps in order and nothing else in the format of a plain JSON array.
Where each item is the step in the format:
{{
  {research_step_key_step_number}: number,
  {research_step_key_step_description}: string,
  {research_step_key_ai_prompt}: string, # each prompt should take into account the possible results from the previous step
  can_be_solved_by_ai: boolean,
  needs_human_input: boolean,
  why_human_input_is_needed: string
}}

Important: do not include things like "```json" or "```" in your response.
"""

if model == 'gemini-1.0-pro' and system_instructions is not None:
  system_instructions = None
  print('\x1b[31m(WARNING: System instructions ignored, gemini-1.0-pro does not support system instructions)\x1b[0m')

if model == 'gemini-1.0-pro' and temperature > 1:
  temperature = 1
  print('\x1b[34m(INFO: Temperature set to 1, gemini-1.0-pro does not support temperature > 1)\x1b[0m')

if system_instructions == '':
  system_instructions = None

model = genai.GenerativeModel(model, system_instruction=system_instructions)
config = genai.GenerationConfig(temperature=temperature, stop_sequences=[])


def generate_ai_prompt_response(prompt_text):
    try:
        response = model.generate_content(contents=[prompt_text], generation_config=config)
        return response.text
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Usage
research_steps_raw = generate_ai_prompt_response(prompt)

if research_steps_raw:
    print("Research steps generated successfully.")
    if "```" in research_steps_raw:
      raise SystemError("AI failed to generate valid plain json response")
else:
    print("Research steps generation failed.")

Research steps generated successfully.


In [51]:
# @title Display steps

print(research_steps_raw)

[
  {
    "step_number": 1,
    "step_description": "Define the scope and objectives of the response paper.  Clearly articulate the specific aspects of \"Automated Investigation and Research Assistants - Towards AI-powered Knowledge Discovery\" that will be addressed.",
    "ai_prompt_for_solving": "Generate three potential scopes for a response paper on \"Automated Investigation and Research Assistants - Towards AI-powered Knowledge Discovery\", each with specific objectives and limitations.",
    "can_be_solved_by_ai": true,
    "needs_human_input": true,
    "why_human_input_is_needed": "AI-generated scopes need human evaluation to ensure relevance, originality, and feasibility within the given constraints (e.g., word count, deadline)."
  },
  {
    "step_number": 2,
    "step_description": "Conduct a comprehensive literature review on AI-powered research assistants, focusing on existing tools, methodologies, and challenges.",
    "ai_prompt_for_solving": "Find and summarize the top

In [45]:
# @title Select step to work on with AI assistant

import ipywidgets as widgets
from IPython.display import display


research_steps_json = json.loads(research_steps_raw)

current_step_selector = widgets.Dropdown(
    options=[(step.get(research_step_key_step_description), idx) for (idx, step) in enumerate(research_steps_json)],
    index=0,
    layout=widgets.Layout(width='100%'),
    description="Step the AI researcher is currently assisting on."
)

display(current_step_selector)

Dropdown(description='Step the AI researcher is currently assisting on.', layout=Layout(width='100%'), options…

In [62]:
# @title Generate output for selected research step

user_guide = f"""
# Output from steps are automatically saved in your session so you can switch between steps at any point.
# However later steps that depend on the output from previous steps will need to be run again
# TODO: make this automatic so that any future run steps are automatically regenerated
# TODO: print why human input is needed
"""

print(user_guide)

step_ai_outputs = dict() # TODO: remove this line as it resets each time the cell is run

current_step_index = current_step_selector.value
current_step = research_steps_json[current_step_index]
user_guide_run_step_no_value = "Run the cell to generate the output"
user_guide_run_step_reprompt = "Generate 5 research questions"
user_guide_run_step_has_value = f"Enter a prompt to modify the most recent AI generated output e.g. '{user_guide_run_step_reprompt}'"

current_step_output = step_ai_outputs.get(current_step_index, {});
step_ai_outputs[current_step_index] = current_step_output # TODO: more "pythonic" syntax

# layout styling for display components
textarea_height = "90%";
textarea_style = {"description_width": "20%"}

# create text field to show original prompt
current_step_display_og_prompt = widgets.Textarea(
    value=current_step.get(research_step_key_ai_prompt),
    description="Original prompt:",
    disabled=True,
    layout=widgets.Layout(height="50px", width=textarea_height),
    style=textarea_style
)

# create text field for reprompt and prepopulate it with the current step existing reprompt value
current_step_output_editor_prompt = widgets.Textarea(
    value=current_step_output.get(current_step_key_reprompt, ""),
    placeholder=f"Example: '{user_guide_run_step_reprompt}'",
    description="Reprompt:",
    disabled=False,
    layout=widgets.Layout(height="50px", width=textarea_height),
    style=textarea_style
)

# update current step output reprompt value based on user input
current_step_output[current_step_key_reprompt] = current_step_output_editor_prompt.value

step_ai_og_prompt = current_step.get(research_step_key_ai_prompt)
step_ai_re_prompt = current_step_output[current_step_key_reprompt]
# TODO: specify a format that can be fed into the the next step ai generation
step_ai_generated_value = current_step_output.get(current_step_key_value)

if (step_ai_generated_value == None):
  # auto generate output if user has not gotten one yet
  current_step_output[current_step_key_value] = generate_ai_prompt_response(step_ai_og_prompt)
elif step_ai_re_prompt:
  # if the user has a reprompt set update the output based on the reprompt
  reprompt_to_submit = f"""
For: {step_ai_og_prompt}
AI replied: '{step_ai_generated_value}'

{step_ai_re_prompt}
"""
  current_step_output[current_step_key_value] = generate_ai_prompt_response(reprompt_to_submit)

# create text field to show the most recent generated ai output
current_step_output_ai_response = widgets.Textarea(
    value=current_step_output.get(current_step_key_value, user_guide_run_step_has_value),
    description="Generated output:",
    disabled=True,
    layout=widgets.Layout(height="400px", width="90%"),
    style=textarea_style
)

display(current_step_display_og_prompt)
display(current_step_output_editor_prompt)
display(current_step_output_ai_response)


# Output from steps are automatically saved in your session so you can switch between steps at any point.
# However later steps that depend on the output from previous steps will need to be run again
# TODO: make this automatic so that any future run steps are automatically regenerated
# TODO: print why human input is needed



Textarea(value='Generate three potential scopes for a response paper on "Automated Investigation and Research …

Textarea(value='', description='Reprompt:', layout=Layout(height='50px', width='90%'), placeholder="Example: '…

Textarea(value='Here are three potential scopes for a response paper on "Automated Investigation and Research …